1. All imports 

In [20]:
import base64
import datetime
import hashlib
import json
import os
import re
import requests
from time import sleep

from requests.auth import AuthBase, HTTPBasicAuth
from requests_oauthlib import OAuth2Session

2. Initialize the main package to read in env, configs etc.

In [21]:
#[2.1] Initialize the main package to read in env, configs etc.
import importlib  
spellbook = importlib.import_module("spellbook-py")

#[2.2] Define Twitter credentials from .env
credentials = {
    "client_id": os.environ["TWITTER_CLIENT_ID"],
    "client_secret": os.environ["TWITTER_CLIENT_SECRET"],
    "redirect_uri": os.environ["TWITTER_REDIRECT_URI"]
    }

print (credentials)

{'client_id': 'Q183X1Zrc3ZxV3JUNlhCMDVFRXQ6MTpjaQ', 'client_secret': 'yI7gzZVtvMXtVXRa0-S7G0uILvyYQJTUUVfdyKQTRtBeSXglLY', 'redirect_uri': 'https://peterpodsky.com/dev/'}


3. Get suthorisation url

In [23]:
#[3] Set the scopes what data will be accessed
# offline.access makes it possible to fetch a new refresh token without prompting a new login session. When the access token has expired, you can fetch a new one with the refresh token. The refresh token is valid for 6 months.

scopes = ["bookmark.read", "tweet.read", "users.read", "offline.access"]

#[4] We now need to perform a PKCE / Proof Key for Code Exchange for OAuth 2.0. Read more e.g. here: https://medium.com/identity-beyond-borders/what-the-heck-is-pkce-40662e801a76
#[4.1] Creating code verifier: cryptographically random string generated by your app
credentials["code_verifier"] = base64.urlsafe_b64encode(os.urandom(30)).decode("utf-8")
credentials["code_verifier"] = re.sub("[^a-zA-Z0-9]+", "", credentials["code_verifier"]) #use only standard signs

#[4.2] Creating code challenge: 
code_challenge = hashlib.sha256(credentials["code_verifier"].encode("utf-8")).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode("utf-8")
code_challenge = code_challenge.replace("=", "")

#[5] Start oauth session:
oauth = OAuth2Session(credentials["client_id"], 
                        redirect_uri=credentials["redirect_uri"], 
                        scope=scopes)
auth_url = "https://twitter.com/i/oauth2/authorize"
credentials["authorization_url"], state = oauth.authorization_url( auth_url, 
                                                    code_challenge=code_challenge,
                                                    code_challenge_method="S256")

print(f'Authorisation URI: {credentials["authorization_url"]}')

Authorisation URI: https://twitter.com/i/oauth2/authorize?response_type=code&client_id=Q183X1Zrc3ZxV3JUNlhCMDVFRXQ6MTpjaQ&redirect_uri=https%3A%2F%2Fpeterpodsky.com%2Fdev%2F&scope=bookmark.read+tweet.read+users.read+offline.access&state=5a7q5b4vL5dhfozoXNihGJHtHKBXQJ&code_challenge=1x2pGPpijya7WP52XIkNuCcJQKXyZejceqZuv_JLFKI&code_challenge_method=S256


4. Get twitter access token, that will allow offline access (When the access token has expired, you can fetch a new one with the refresh token. The refresh token is valid for 6 months.) 

In [24]:
authorization_response = "https://peterpodsky.com/dev/?state=5a7q5b4vL5dhfozoXNihGJHtHKBXQJ&code=dzJ2WGwtcWJteEZJYjhrd0doek52MTZfUkVIYlB5UmpWRzhnWmxYQjZkemYtOjE3MDAxMzY5ODgzOTc6MToxOmFjOjE"

token_url = "https://api.twitter.com/2/oauth2/token"
auth = HTTPBasicAuth(credentials["client_id"], credentials["client_secret"])

credentials["token"] = oauth.fetch_token(
    token_url=token_url,
    authorization_response=authorization_response,
    auth=auth,
    client_id=credentials["client_id"],
    include_client_id=True,
    code_verifier=credentials["code_verifier"],
)
print(f'Token: {credentials["token"]}')

Token: {'token_type': 'bearer', 'expires_in': 7200, 'access_token': 'dUNBUnk3WVFoVmNkQldnanRGbWlnSEdycTNDMjFhYnFwemtuR09ubEkxR0FqOjE3MDAxMzY5OTYwMTQ6MTowOmF0OjE', 'scope': ['users.read', 'tweet.read', 'offline.access', 'bookmark.read'], 'refresh_token': 'blpLTkVXSDR6b2YtdUNZSjA1NFJOVU5TbHZsSkppWWlNUEdzRTdSejkxdzhtOjE3MDAxMzY5OTYwMTQ6MTowOnJ0OjE', 'expires_at': 1700144196.203476}


5. Get user ID for a current access token from Twitter

In [25]:
user_me = requests.get(
    "https://api.twitter.com/2/users/me",
    headers={"Authorization": f'Bearer {credentials["token"]["access_token"]}'},
).json()

credentials["user_id"] = user_me["data"]["id"]
print(f'user_id: {credentials["user_id"]}')

user_id: 1215694754


6.  Call the bookmark API endpoint and fetch  bookmarks via HTTP GET call to API:



In [26]:
url = f"https://api.twitter.com/2/users/{credentials['user_id']}/bookmarks"
headers = {
    "Authorization": f"Bearer {credentials['token']['access_token']}",
}
response = requests.get(url, headers=headers, params={
    'tweet.fields':'author_id,created_at',
    'expansions':'author_id',
    'user.fields':'username',
})

print(f'Response: {response.json()}')
tweets = response.json()['data']
print(f'Response: {response.json().keys()}')
print(f'pagination_token : {response.json()["next_token"]}')
print(f'Bookmark #1: {response.json()["data"][0]}')
print(f'User #1: {response.json()["includes"]["users"]}')

Response: {'client_id': '28054117', 'detail': 'When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.', 'registration_url': 'https://developer.twitter.com/en/docs/projects/overview', 'title': 'Client Forbidden', 'required_enrollment': 'Appropriate Level of API Access', 'reason': 'client-not-enrolled', 'type': 'https://api.twitter.com/2/problems/client-forbidden'}


KeyError: 'data'